## Initial setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
print(tf.__version__)
import torch
print(torch.__version__)
import matplotlib
print(matplotlib.__version__)

2.8.0
1.10.0+cu111
3.2.2


In [ ]:
!nvidia-smi

Fri Jan 21 19:41:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Other imports
! pip install tensorflow_addons
! pip install tensorflow_io

import os
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator
from imutils import paths
from tqdm import tqdm
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import tensorflow_io as tfio
import tensorflow_hub as hub
import numpy as np
import cv2
import pandas as pd
import seaborn as sns
from scipy.stats import mannwhitneyu
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
import sklearn.manifold
from sklearn.metrics.pairwise import cosine_similarity as cos
from sympy.utilities.iterables import multiset_permutations
from sklearn.metrics import accuracy_score, f1_score,precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import *
from sklearn.preprocessing import StandardScaler
from IPython.display import Image, display


import zipfile
import concurrent.futures

# Random seed fix
random_seed = 42
tf.random.set_seed(random_seed)
np.random.seed(random_seed)

     |████████████████████████████████| 1.1 MB 6.4 MB/s 
     |████████████████████████████████| 23.4 MB 1.8 MB/s 


## Dataset gathering and preparation

In [ ]:
%cd /content/drive/MyDrive/nanowire-morphology-classification-project

In [ ]:
training_batch_size = 4

BATCH_SIZE = training_batch_size

imageSize = 224

category_names = ['bundle', 'dispersed', 'network', 'singular']
color_method = ['C0', 'C1', 'C2', 'C3', 'C4']
color = ['black', 'magenta', 'cyan', 'yellow']
marker = ['o', 's', '<', '>', '^']
seaborn_palette = sns.color_palette("colorblind")

In [ ]:
# Image preprocessing utils
@tf.function
def parse_images(image_path):
    image_string = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image_string, channels=3)
    # image = tfio.experimental.image.decode_tiff(image_string)[:, :, :3]   # in the doc, it transforms tiff to 4 channels, with additional channel of opacity which is not needed.
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, size=[imageSize, imageSize])

    return image

## clustering and percolation analysis of the segmentation ground truth

In [ ]:
np.random.seed(random_seed)
peptide_morph_seglabel_train_path = "/content/drive/MyDrive/TEM image datasets/2022-nanowire-morphology"
peptide_morph_images_train_seglabel = list(paths.list_files(basePath=peptide_morph_seglabel_train_path, validExts='png'))
peptide_morph_images_train_seglabel = np.random.choice(np.array(peptide_morph_images_train_seglabel), len(peptide_morph_images_train_seglabel), replace=False)
print(len(peptide_morph_images_train_seglabel))

200


In [ ]:
def generate_ground_truth_images(image, resolution):
  image_bool = np.ones((resolution, resolution))
  for i in range(image.shape[0]):
    for j in range(image.shape[1]):
      if image[i, j, 1] == image[i, j, 2]:
        image_bool[i, j] = 0            # background is black with code of 0
      else: 
        image_bool[i, j] = 1            # nanowire is white with code of 1
  return image_bool

In [ ]:
segmentation_class_labels = []
for i in range(peptide_morph_images_train_seglabel.shape[0]):
  seg_class_label = peptide_morph_images_train_seglabel[i].split("/")[-2]
  segmentation_class_labels.append(seg_class_label)
le = LabelEncoder()
peptide_morph_train_seg_enc = le.fit_transform(segmentation_class_labels)

In [ ]:
# image_mask = np.zeros((len(peptide_morph_images_train_seglabel), imageSize, imageSize))
images_no_annotation_directory = []
images_no_annotation = np.zeros((len(peptide_morph_images_train_seglabel), imageSize, imageSize, 3))

for i in range(len(peptide_morph_images_train_seglabel)):
  images_no_annotation_directory.append('%s.jpg' % (peptide_morph_images_train_seglabel[i].split("_")[0] + "_" + peptide_morph_images_train_seglabel[i].split("_")[1]))
  images_no_annotation[i] = parse_images(images_no_annotation_directory[i])

  # these were used to create the ground truth grayscale images from the manual segmentation labels.
  # image_string = tf.io.read_file(peptide_morph_images_train_seglabel[i])
  # image = tf.image.decode_image(image_string, channels=3) / 255
  # image = tf.image.resize(image, (imageSize, imageSize))
  # image = tf.image.convert_image_dtype(image, tf.float32)
  # trans_nd_image_array = image.numpy()
  # image_mask[i] = generate_ground_truth_images(trans_nd_image_array, imageSize)
  
# np.savez_compressed('seg_mask_res512.npz', mask=image_mask)

# once we have the seg_mask saved, we can directly load from npz file
image_mask = np.load('seg_mask_res%i.npz' % (imageSize), allow_pickle=True)['mask']

In [ ]:
# quantify percolation or not for each segmentation mask from the two morphologies
from sklearn.cluster import DBSCAN
nanowire_perc = np.zeros((image_mask.shape[0]))
for k in range(image_mask.shape[0]):
  nanowire_pix = np.transpose(np.vstack((np.where(image_mask[k] == 1)[0], np.where(image_mask[k] == 1)[1])))
  cluster = DBSCAN(eps=2, min_samples=2, n_jobs=-1).fit(nanowire_pix)
  cluster_size = np.zeros(np.unique(cluster.labels_).shape)
  labels = cluster.labels_
  
  for i in np.unique(cluster.labels_):
    if i == -1:            # i == -1 is the default label for points not in any cluster, these are excluded, as they are likely artifacts during resizing the image to lower resolution
      labels = np.delete(labels, np.where(labels == -1)[0])
      continue             # because the segmentation image masks practically don't have single pixels, i.e. they should have close neighbors because they are hand drawn

    cluster_size[i] = np.where(labels == i)[0].shape[0]
    right = np.max(cluster.components_[np.where(labels == i)[0], 0])
    top = np.max(cluster.components_[np.where(labels == i)[0], 1])
    left = np.min(cluster.components_[np.where(labels == i)[0], 0])
    bottom = np.min(cluster.components_[np.where(labels == i)[0], 1])

    count_horizontal = right - left
    count_vertical = top - bottom
    if count_horizontal >= imageSize - 2 and count_vertical >= imageSize - 2:
      nanowire_perc[k] = 1

print('number of dispersed morphology images: %i' % len(np.where(nanowire_perc == 0)[0]))
print('number of percolated network morphology images: %i' % int(sum(nanowire_perc)))

accuracy_score(nanowire_perc, peptide_morph_train_seg_enc)
# if the accuracy_score is 1.0, that means all the dispersed morphology images are validated as not percolated, 
# all the network morphology images are validated as percolated.

number of dispersed morphology images: 100
number of percolated network morphology images: 100


1.0